<a href="https://colab.research.google.com/github/ruijing-xiong/Spoon-Knife/blob/main/codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#loading necessary libraries

In [ ]:
# Install packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set(style="darkgrid")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
import math
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.spatial.distance import cdist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, RationalQuadratic as RQ, WhiteKernel, ExpSineSquared as Exp, DotProduct as Lin


#Step1: Get inputs from google sheets

In [ ]:
#function to get user input values
def get_inputs(worksheet):
  
  #select the second column
  inputs = worksheet.col_values(2)

  #first row is empty

  ##Get crop type and model number from google sheets
  crop_type = inputs[1]
  model_num = inputs[2]
  print(f"crop type: {crop_type}, model num: {model_num}")

  ##Get location from google sheets
  country= inputs[3]

  try:
      city = inputs[4]
  except IndexError as error:
      city =''

  try:
      zipcode = inputs[5]
  except IndexError as error:
      zipcode =''
      
  print(f"city: {city},zipcode: {zipcode}, country: {country}")

  return (city,zipcode, country, crop_type,model_num)

#Step 2: Location indicators

Converting location to longtitude and latitude   



In [ ]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
   
# function to find the coordinate of a given city 
def findGeocode(city,zipcode,country):

  #specify state to avoid the same city name in different states
  if country == 'United States' :
    location = zipcode+ ' '+country

  else: 
    location = city+' '+country

    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
  try:
      geolocator = Nominatim(user_agent="your_app_name")
      loc = geolocator.geocode(location)

      print("address:", loc, "\nlatitude:",loc.latitude, "\nlongitude:",loc.longitude)
      return (loc.latitude,loc.longitude)
      

  except GeocoderTimedOut:
      return findGeocode(city,zipcode,country)    
  
  

## Weather parameters by using API from open-meteo.com

In [ ]:

#function to find weather based on the coordinate of a given city by using API
def weather_parameter(df, longitude,latitude):

  import requests
  import json
  
  ###Use this one units are SI: pressure in hPa, prcp is in mm, wspd in km/hr and temp in C
  response=requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true&hourly=surface_pressure,precipitation&forecast_days=1") # data from METEO

  weather=json.loads(response.text)
  
  #get current weather
  specific_datetime = weather['current_weather']['time']  
  index = weather['hourly']['time'].index(specific_datetime)

  #store 4 weather parameters
  curr_pres = weather['hourly']['surface_pressure'][index]
  curr_prcp = weather['hourly']['precipitation'][index]
  curr_temp = weather['current_weather']['temperature']
  curr_wspd = weather['current_weather']['windspeed']
  
  current_weather  = [curr_prcp,curr_wspd,curr_pres,curr_temp]
  print(f"original value: curr_prcp': {curr_prcp}, curr_temp:{curr_temp},\
  curr_wspd:{curr_wspd}, curr_pres:{curr_pres}")
  
  #weather range
  f = df['curr_prcp']
  g = df['curr_wspd']
  h = df['curr_pres']
  i = df['curr_temp']

#make sure weather data is within the range of the existing dataset

  min_range = {'curr_prcp':np.min(f),'curr_wspd':np.min(g),'curr_pres':np.min(h),'curr_temp':np.min(i)}
  max_range = {'curr_prcp':np.max(f),'curr_wspd':np.max(g),'curr_pres':np.max(h),'curr_temp':np.max(i)}

#curr_prcp:
#  message = ''
  if curr_prcp < min_range.get('curr_prcp'):
#    message = 'current prcp'+curr_prcp+' is out of range，min value '+ min_range.get('curr_prcp') +' is applied'
    curr_prcp = round(min_range.get('curr_prcp'))
  if curr_prcp > max_range.get('curr_prcp'):
#    message = 'current prcp'+curr_prcp+' is out of range，max value '+ max_range.get('curr_prcp') +' is applied'
    curr_prcp = round(max_range.get('curr_prcp'))
  else: 
    curr_prcp
#    message

#curr_wspd:
  if curr_wspd < min_range.get('curr_wspd'):
#    message = 'current prcp'+curr_prcp+' is out of range，min value '+ min_range.get('curr_prcp') +' is applied'
    curr_wspd = round(min_range.get('curr_wspd'))
  if curr_wspd > max_range.get('curr_wspd'):
#    message = 'current prcp'+curr_prcp+' is out of range，max value '+ max_range.get('curr_prcp') +' is applied'
    curr_wspd = round(max_range.get('curr_wspd'))
  else: 
    curr_wspd

#curr_pres:
  if curr_pres < min_range.get('curr_pres'):
#    message = 'current prcp'+curr_prcp+' is out of range，min value '+ min_range.get('curr_prcp') +' is applied'
    curr_pres = round(min_range.get('curr_pres'))
  if curr_pres > max_range.get('curr_pres'):
#    message = 'current prcp'+curr_prcp+' is out of range，max value '+ max_range.get('curr_prcp') +' is applied'
    curr_pres = round(max_range.get('curr_pres'))
  else: 
    curr_pres

#curr_temp:
  if curr_temp < min_range.get('curr_temp'):
#    message = 'current prcp'+curr_prcp+' is out of range，min value '+ min_range.get('curr_prcp') +' is applied'
    curr_temp = round(min_range.get('curr_temp'))
  if curr_temp > max_range.get('curr_temp'):
#    message = 'current prcp'+curr_prcp+' is out of range，max value '+ max_range.get('curr_prcp') +' is applied'
    curr_temp = round(max_range.get('curr_temp'))
  else: 
    curr_temp

  print(f"curr_prcp': {curr_prcp}, curr_temp:{curr_temp},\
  curr_wspd:{curr_wspd}, curr_pres:{curr_pres}")
  
  return curr_prcp,curr_wspd,curr_pres,curr_temp

#Step 3: clean and slicing data

In [ ]:
def clean(df,crop_type,model_num):
  
  from sklearn.preprocessing import StandardScaler

  #get target variables
  df['delta'] = (df['broken_grain_actual']+df['mog_actual']-df['yield'])

  # Creating an instance of the sklearn.preprocessing.MinMaxScaler()
  scaler = StandardScaler()

  #scale the data
  df[["ScaledBG", "ScaledMOG", "ScaledYield"]] = scaler.fit_transform(df[["broken_grain_actual","mog_actual", "yield"]])
  
  #create scalled target variables
  df["ScaledDelta"] = df["ScaledBG"] + df["ScaledMOG"] - df["ScaledYield"]
  df["ScaledDelta"]  

  #drop unneccessary columns
  df1 = df.drop(['hist_pres', 'hist_wspd', 'hist_prcp', 'hist_temp', 'timestamp',\
                 'telemetry_date', 'latitude', 'longitude'], axis=1)
  df2 = df1.drop(['broken_grain_goal', 'mog_goal', 'yield', 'delta', 'broken_grain_actual', 'mog_actual', 'ScaledBG', 'ScaledMOG', 'ScaledYield', 'grain_loss_rotor', 'grain_loss_shoe', 'serial_num'], axis=1)
  
  # replacing WET CORN and DRY CORN to CORN
  df2['crop_type'].mask(df2['crop_type'] == 'DRY CORN', 'CORN', inplace=True)
  df2['crop_type'].mask(df2['crop_type'] == 'WET CORN', 'CORN', inplace=True)

  # Merge combine 7 and 8
  df2['model_num'] = df2['model_num'].replace(7, 8)
  df2['model_num'] = df2['model_num'].replace(7, 8)
  df2['model_num'] = df2['model_num'].replace(7, 8)

  # Merge combine 9 and 10
  df2['model_num'] = df2['model_num'].replace(9, 10)
  df2['model_num'] = df2['model_num'].replace(9, 10)
  df2['model_num'] = df2['model_num'].replace(9, 10)

  # slicing data frame based on 3 crop types 'WHEAT', 'CORN', 'SOYBEAN'.
  # variable to hold input data frame
  in_df = df2

  #identify crop df
  #if matches with the input values
  crop_df = in_df.loc[(in_df['crop_type'] == crop_type)]

  # identify model_num df 
  if '8' in model_num:
    new_df = crop_df.loc[(crop_df['model_num'] == 8)]

  if '10' in model_num:
    new_df = crop_df.loc[(crop_df['model_num'] == 10)]

  # drop columns
  new_df = new_df.drop(['crop_type', 'model_num'], axis=1)
  new_df = new_df.dropna()

  #default settings
  default_corn={'cleaning_fan_speed':950,'chaffer_position':20, 'sieve_position':16,'rotor_speed':400, 'concave_position':22}
  default_soybeans={'cleaning_fan_speed':950,'chaffer_position':18, 'sieve_position':14,'rotor_speed':650, 'concave_position':14}
  default_wheat={'cleaning_fan_speed':850,'chaffer_position':16, 'sieve_position':12,'rotor_speed':850, 'concave_position':12}

  setting = {'CORN':default_corn,'SOYBEANS':default_soybeans, 'WHEAT':default_wheat}

  default_setting = setting.get(crop_type)


  return new_df, default_setting
  


#Step 4: Modeling

**Set up weather parameters**

In [ ]:
def GaussianProcessRegressor(df,curr_prcp,curr_temp,curr_wspd,curr_pres,default_setting):
  import numpy as np
  import matplotlib.pyplot as plt
  from mpl_toolkits.mplot3d import Axes3D
  from sklearn.gaussian_process import GaussianProcessRegressor
  from sklearn.gaussian_process.kernels import RBF
  import scipy.optimize as opt
  from scipy.optimize import LbfgsInvHessProduct
  import numpy as np
  from scipy.optimize import minimize

  # Create scatter plot
  a = df['cleaning_fan_speed']
  b = df['chaffer_position']
  c = df['sieve_position']
  d = df['rotor_speed']
  e = df['concave_position']
  f = df['curr_prcp']
  g = df['curr_wspd']
  h = df['curr_pres']
  i = df['curr_temp']
  z = df['ScaledDelta']

  # Define kernel for Gaussian Process Regressor
  kernel = RBF()

  # Create Gaussian Process Regressor object
  gpr = GaussianProcessRegressor(kernel=RBF())

  # Fit the Gaussian Process Regressor to the scatter plot data
  X = np.vstack((a, b, c, d, e, f, g, h, i)).T
  gpr.fit(X, z)

  # Interpolate new x and y values using the Gaussian Process Regressor
  new_fan = np.arange(default_setting.get('cleaning_fan_speed')-150,151+default_setting.get('cleaning_fan_speed'),50)
  new_chaffer = np.arange(default_setting.get('chaffer_position')-8,9+default_setting.get('chaffer_position'))
  new_sieve = np.arange(default_setting.get('sieve_position')-6,7+default_setting.get('sieve_position'))
  new_rotor = np.arange(default_setting.get('rotor_speed')-150,151+default_setting.get('rotor_speed'),50)
  new_concave = np.arange(default_setting.get('concave_position')-10,11+default_setting.get('concave_position'))
  new_f = round(curr_prcp)
  new_g = round(curr_wspd)
  new_h = round(curr_pres)
  new_i = round(curr_temp)
  X_new = np.meshgrid(new_fan, new_chaffer, new_sieve, new_rotor, new_concave, new_f, new_g, new_h, new_i)
  X_new_2d = np.array((X_new[0].flatten(), X_new[1].flatten(), X_new[2].flatten(),X_new[3].flatten(),X_new[4].flatten(),X_new[5].flatten(), X_new[6].flatten(), X_new[7].flatten(), X_new[8].flatten())).T

  z_pred1, sigma = gpr.predict(X_new_2d, return_std=True)

  # find coords of min in z_pred
  min_idx = np.argmin(z_pred1)
  print(f'min delta: {z_pred1[min_idx]}')

  results =[str(round(X_new_2d[min_idx,0])),str(round(X_new_2d[min_idx,1])),\
            str(round(X_new_2d[min_idx,2])),str(round(X_new_2d[min_idx,3])), \
            str(round(X_new_2d[min_idx,4]))]
  print(f'ideal values: \ncleaning_fan_speed: {round(X_new_2d[min_idx,0])},\
          \nchaffer_position: {round(X_new_2d[min_idx,1])},\
          \nsieve_position: {round(X_new_2d[min_idx,2])},\
          \nrotor_speed: {round(X_new_2d[min_idx,3])},\
          \nconcave_position: {round(X_new_2d[min_idx,4])},\
          \ncurr_prcp: {round(X_new_2d[min_idx,5])},\
          \ncurr_wspd: {round(X_new_2d[min_idx,6])},\
          \ncurr_pres: {round(X_new_2d[min_idx,7])},\
          \ncurr_temp: {round(X_new_2d[min_idx,8])}')
  return results

#Step 5: Update Google sheets and display recommended settings

In [ ]:

##function to update google sheets with recommended settings
def update(worksheet,results):
  #update google sheet
  cell_list = worksheet.range('E2:E6')
  #warning_list = worksheet.range('E7:E10')
  #gives us a tuple of an index and value
  for i, val in enumerate(results):  
  #use the index on cell_list and the val from cell_values
      cell_list[i].value = val    

  worksheet.update_cells(cell_list)

#Main Command

##connect with database

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('combines_data.csv')
df.head(5)

In [ ]:
from google.colab import auth
auth.authenticate_user()

**open and read values on google sheets**

In [ ]:
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('combine_settings_inputs').sheet1

##running models

**reading inputs from google sheets**

In [ ]:
city,zipcode,country,crop_type,model_num = get_inputs(worksheet)

crop type: CORN, model num: 7-8
city: ,zipcode: 61701, country: United States


**converting address to latitude and longtitude**

In [ ]:
latitude, longitude= findGeocode(city,zipcode,country)

address: Bloomington, McLean County, Illinois, 61701, United States 
latitude: 40.47972329777424 
longitude: -88.98931991081082


**getting weather parameters from website**

In [ ]:
curr_prcp,curr_temp,curr_wspd,curr_pres = weather_parameter(df, longitude,latitude)

original value: curr_prcp': 0.0, curr_temp:13.2,  curr_wspd:8.0, curr_pres:984.2
curr_prcp': 0.0, curr_temp:13.2,  curr_wspd:8.0, curr_pres:1005


**clean data and make it ready to run models**

In [ ]:
processed_data,default_setting = clean(df,crop_type,model_num)

**getting outputs from Gussian Process Regressor**

In [ ]:
#RBF
output = GaussianProcessRegressor(processed_data,curr_prcp,curr_temp,curr_wspd,curr_pres,default_setting)

**updating google sheet**

In [ ]:
update(worksheet,output)
